In [1]:
import pandas as pd
import numpy as np
import requests


In [ ]:
# automate retreival of data
url = 'https://www.nhc.noaa.gov/data/hurdat/'
filename = 'hurdat2-1851-2024-040425.txt'
txt_data = requests.get(url + filename)

with open(filename, "w") as f:
    f.write(txt_data.text)


In [9]:
# Step 1: Load data

with open(filename, 'r') as f:
    lines = f.readlines()

df_raw = pd.DataFrame(lines, columns=['raw_line'])
df_raw['raw_line'] = df_raw['raw_line'].str.strip()  # Remove \n

# Debugging outputs
print(f"Total lines in file: {len(df_raw)}")
print("\nFirst 5 lines:")
print(df_raw.head())

Total lines in file: 57221

First 5 lines:
                                            raw_line
0              AL011851,            UNNAMED,     14,
1  18510625, 0000,  , HU, 28.0N,  94.8W,  80, -99...
2  18510625, 0600,  , HU, 28.0N,  95.4W,  80, -99...
3  18510625, 1200,  , HU, 28.0N,  96.0W,  80, -99...
4  18510625, 1800,  , HU, 28.1N,  96.5W,  80, -99...


In [8]:

## Step 2: Create and populate the is_header column
df_raw['is_header'] = df_raw['raw_line'].str.startswith('AL')

print("\nHeader lines found:", df_raw['is_header'].sum())
print("Data lines found:", (~df_raw['is_header']).sum())
print("\nFirst 5 header lines:")
print(df_raw[df_raw['is_header']].head())




Header lines found: 1991
Data lines found: 55230

First 5 header lines:
                                 raw_line  is_header
0   AL011851,            UNNAMED,     14,       True
15  AL021851,            UNNAMED,      1,       True
17  AL031851,            UNNAMED,      1,       True
19  AL041851,            UNNAMED,     49,       True
69  AL051851,            UNNAMED,     16,       True


In [4]:

# Step 3: Extract storm info from header rows
df_raw['storm_id'] = df_raw[df_raw['is_header']]['raw_line'].str.split(',').str[0]
df_raw['storm_name'] = df_raw[df_raw['is_header']]['raw_line'].str.split(',').str[1].str.strip()
df_raw['best_track_entry'] = df_raw[df_raw['is_header']]['raw_line'].str.split(',').str[2].str.strip()

print("\nFirst 5 storm headers:")
print(df_raw[df_raw['is_header']][['storm_id', 'storm_name', 'best_track_entry']].head())


First 5 storm headers:
    storm_id storm_name best_track_entry
0   AL011851    UNNAMED               14
15  AL021851    UNNAMED                1
17  AL031851    UNNAMED                1
19  AL041851    UNNAMED               49
69  AL051851    UNNAMED               16


In [5]:
# Step 4: Populate storm info into data rows now all lines have storm name and id

    # I coded this first and got this warning FutureWarning: Series.fillna with 'method' is deprecated 
    # and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

    # df_raw['storm_id'] = df_raw['storm_id'].fillna(method='ffill')
    # df_raw['storm_name'] = df_raw['storm_name'].fillna(method='ffill')

df_raw['storm_id'] = df_raw['storm_id'].ffill()
df_raw['storm_name'] = df_raw['storm_name'].ffill()
df_raw['best_track_entry'] = df_raw['best_track_entry'].ffill()

print("\nAfter forward fill, first 5 rows:")
print(df_raw.head())



After forward fill, first 5 rows:
                                            raw_line  is_header  storm_id  \
0              AL011851,            UNNAMED,     14,       True  AL011851   
1  18510625, 0000,  , HU, 28.0N,  94.8W,  80, -99...      False  AL011851   
2  18510625, 0600,  , HU, 28.0N,  95.4W,  80, -99...      False  AL011851   
3  18510625, 1200,  , HU, 28.0N,  96.0W,  80, -99...      False  AL011851   
4  18510625, 1800,  , HU, 28.1N,  96.5W,  80, -99...      False  AL011851   

  storm_name best_track_entry  
0    UNNAMED               14  
1    UNNAMED               14  
2    UNNAMED               14  
3    UNNAMED               14  
4    UNNAMED               14  


In [6]:

# Step 5: Filter to only data rows (boolean indexing)
print("BEFORE filtering:")
print(f"Total rows in df_raw: {len(df_raw)}")
print(f"Header rows: {df_raw['is_header'].sum()}")
print(f"Data rows: {(~df_raw['is_header']).sum()}")
print("\nSample of what we're filtering out (header rows):")
print(df_raw[df_raw['is_header']]['raw_line'].head(3))

df_data = df_raw[~df_raw['is_header']].copy()

# Additional validation: ensure all rows start with 8-digit dates
first_element = df_data['raw_line'].str.split(',', expand=True)[0].str.strip()
is_valid_date = first_element.str.match(r'^\d{8}$')

if (~is_valid_date).any():
    print(f"\n Found {(~is_valid_date).sum()} rows without valid dates and removing them ")
    df_data = df_data[is_valid_date].copy()

print(f"\nAFTER filtering:")
print(f"Total data rows: {len(df_data)}")
print(f"is_header column values in df_data: {df_data['is_header'].unique()}")
print("\nfirst 3 data rows kept:")
print(df_data[['raw_line', 'storm_id', 'storm_name']].head())


BEFORE filtering:
Total rows in df_raw: 57221
Header rows: 1991
Data rows: 55230

Sample of what we're filtering out (header rows):
0     AL011851,            UNNAMED,     14,
15    AL021851,            UNNAMED,      1,
17    AL031851,            UNNAMED,      1,
Name: raw_line, dtype: object

AFTER filtering:
Total data rows: 55230
is_header column values in df_data: [False]

first 3 data rows kept:
                                            raw_line  storm_id storm_name
1  18510625, 0000,  , HU, 28.0N,  94.8W,  80, -99...  AL011851    UNNAMED
2  18510625, 0600,  , HU, 28.0N,  95.4W,  80, -99...  AL011851    UNNAMED
3  18510625, 1200,  , HU, 28.0N,  96.0W,  80, -99...  AL011851    UNNAMED
4  18510625, 1800,  , HU, 28.1N,  96.5W,  80, -99...  AL011851    UNNAMED
5  18510625, 2100, L, HU, 28.2N,  96.8W,  80, -99...  AL011851    UNNAMED


In [7]:

# Step 6: Split the raw_line into columns
print(f"STEP 6 - Input: {df_data.shape[0]} rows with 1 'raw_line' column")
data_columns = df_data['raw_line'].str.split(',', expand=True)
print(f"STEP 6 - Output: {data_columns.shape[0]} rows with {data_columns.shape[1]} columns")

df_data.head()


STEP 6 - Input: 55230 rows with 1 'raw_line' column
STEP 6 - Output: 55230 rows with 21 columns
STEP 6 - Output: 55230 rows with 21 columns


,raw_line,is_header,storm_id,storm_name,best_track_entry
1,"18510625, 0000, , HU, 28.0N, 94.8W, 80, -99...",False,AL011851,UNNAMED,14
2,"18510625, 0600, , HU, 28.0N, 95.4W, 80, -99...",False,AL011851,UNNAMED,14
3,"18510625, 1200, , HU, 28.0N, 96.0W, 80, -99...",False,AL011851,UNNAMED,14
4,"18510625, 1800, , HU, 28.1N, 96.5W, 80, -99...",False,AL011851,UNNAMED,14
5,"18510625, 2100, L, HU, 28.2N, 96.8W, 80, -99...",False,AL011851,UNNAMED,14


In [8]:

# Step 7: Combine with storm info (pd.concat)
df_clean = pd.concat([
    df_data[['storm_id', 'storm_name', 'best_track_entry']],  # Don't reset index, both data frames need matching indices
    data_columns                           # Don't reset index
], axis=1)

print(f"STEP 7 - df_clean shape: {df_clean.shape}")
df_clean.head()


STEP 7 - df_clean shape: (55230, 24)


,storm_id,storm_name,best_track_entry,0,1,2,3,4,5,6,...,11,12,13,14,15,16,17,18,19,20
1,AL011851,UNNAMED,14,18510625,0000,,HU,28.0N,94.8W,80,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,AL011851,UNNAMED,14,18510625,0600,,HU,28.0N,95.4W,80,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3,AL011851,UNNAMED,14,18510625,1200,,HU,28.0N,96.0W,80,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
4,AL011851,UNNAMED,14,18510625,1800,,HU,28.1N,96.5W,80,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
5,AL011851,UNNAMED,14,18510625,2100,L,HU,28.2N,96.8W,80,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999


In [9]:

# Step 8: Rename columns (from cheatsheet: rename)
column_names = {
    0: 'date',
    1: 'time', 
    2: 'record_identifier',
    3: 'storm_status',
    4: 'latitude',
    5: 'longitude',
    6: 'max_sustained_wind_knots',
    7: 'min_pressure',
    8: 'wind_34kt_ne',
    9: 'wind_34kt_se',
    10: 'wind_34kt_sw',
    11: 'wind_34kt_nw',
    12: 'wind_50kt_ne',
    13: 'wind_50kt_se',
    14: 'wind_50kt_sw',
    15: 'wind_50kt_nw',
    16: 'wind_64kt_ne',
    17: 'wind_64kt_se',
    18: 'wind_64kt_sw',
    19: 'wind_64kt_nw',
    20: 'max_wind_radius'
}
df_clean = df_clean.rename(columns=column_names)

df_clean.head()



,storm_id,storm_name,best_track_entry,date,time,record_identifier,storm_status,latitude,longitude,max_sustained_wind_knots,...,wind_34kt_nw,wind_50kt_ne,wind_50kt_se,wind_50kt_sw,wind_50kt_nw,wind_64kt_ne,wind_64kt_se,wind_64kt_sw,wind_64kt_nw,max_wind_radius
1,AL011851,UNNAMED,14,18510625,0000,,HU,28.0N,94.8W,80,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,AL011851,UNNAMED,14,18510625,0600,,HU,28.0N,95.4W,80,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
3,AL011851,UNNAMED,14,18510625,1200,,HU,28.0N,96.0W,80,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
4,AL011851,UNNAMED,14,18510625,1800,,HU,28.1N,96.5W,80,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
5,AL011851,UNNAMED,14,18510625,2100,L,HU,28.2N,96.8W,80,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999


In [10]:
# Step 9: removing whitespace from string columns
string_columns = ['storm_id', 'storm_name', 'record_identifier', 'storm_status', 'latitude', 'longitude']
for col in string_columns:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].str.strip()
        
# Note to self why I needed to do this:
# because when I split the raw_line into columns, any leading or trailing spaces in those fields
# would have been retained. Stripping whitespace ensures that string comparisons and analyses
# on these columns are accurate and not affected by extraneous spaces.



In [11]:
# Step 10: Making lat and lon workable by adding columns in my cheatsheet is str.extract and replace
# Extract numeric part and hemisphere
df_clean['lat_value'] = df_clean['latitude'].str.extract(r'(\d+\.?\d*)')[0].astype(float)
df_clean['lat_hemisphere'] = df_clean['latitude'].str.extract(r'([NS])')[0]
df_clean['lon_value'] = df_clean['longitude'].str.extract(r'(\d+\.?\d*)')[0].astype(float) 
df_clean['lon_hemisphere'] = df_clean['longitude'].str.extract(r'([EW])')[0]

print("\nExtracted latitude and longitude values:")
print(df_clean[['latitude', 'lat_value', 'lat_hemisphere', 'longitude', 'lon_value', 'lon_hemisphere']].head())


Extracted latitude and longitude values:
  latitude  lat_value lat_hemisphere longitude  lon_value lon_hemisphere
1    28.0N       28.0              N     94.8W       94.8              W
2    28.0N       28.0              N     95.4W       95.4              W
3    28.0N       28.0              N     96.0W       96.0              W
4    28.1N       28.1              N     96.5W       96.5              W
5    28.2N       28.2              N     96.8W       96.8              W


In [12]:
# Apply negative for South and West (from cheatsheet: loc and boolean indexing) 
# ask team if this is the right format for mapping software, I checked and it is 
df_clean.loc[df_clean['lat_hemisphere'] == 'S', 'lat_value'] *= -1
df_clean.loc[df_clean['lon_hemisphere'] == 'W', 'lon_value'] *= -1

print("\nAfter applying hemisphere adjustments:")
print(df_clean[['latitude', 'lat_value', 'lat_hemisphere', 'longitude', 'lon_value', 'lon_hemisphere']].head())


After applying hemisphere adjustments:
  latitude  lat_value lat_hemisphere longitude  lon_value lon_hemisphere
1    28.0N       28.0              N     94.8W      -94.8              W
2    28.0N       28.0              N     95.4W      -95.4              W
3    28.0N       28.0              N     96.0W      -96.0              W
4    28.1N       28.1              N     96.5W      -96.5              W
5    28.2N       28.2              N     96.8W      -96.8              W


In [13]:

# Replace original columns
df_clean['latitude'] = df_clean['lat_value']
df_clean['longitude'] = df_clean['lon_value']

df_clean.head()

,storm_id,storm_name,best_track_entry,date,time,record_identifier,storm_status,latitude,longitude,max_sustained_wind_knots,...,wind_50kt_nw,wind_64kt_ne,wind_64kt_se,wind_64kt_sw,wind_64kt_nw,max_wind_radius,lat_value,lat_hemisphere,lon_value,lon_hemisphere
1,AL011851,UNNAMED,14,18510625,0000,,HU,28.0,-94.8,80,...,-999,-999,-999,-999,-999,-999,28.0,N,-94.8,W
2,AL011851,UNNAMED,14,18510625,0600,,HU,28.0,-95.4,80,...,-999,-999,-999,-999,-999,-999,28.0,N,-95.4,W
3,AL011851,UNNAMED,14,18510625,1200,,HU,28.0,-96.0,80,...,-999,-999,-999,-999,-999,-999,28.0,N,-96.0,W
4,AL011851,UNNAMED,14,18510625,1800,,HU,28.1,-96.5,80,...,-999,-999,-999,-999,-999,-999,28.1,N,-96.5,W
5,AL011851,UNNAMED,14,18510625,2100,L,HU,28.2,-96.8,80,...,-999,-999,-999,-999,-999,-999,28.2,N,-96.8,W


In [14]:

# Drop helper columns 
df_clean = df_clean.drop(columns=['lat_value', 'lat_hemisphere', 'lon_value', 'lon_hemisphere'])



In [15]:

# Step 11: Convert to numeric (from cheatsheet: pd.to_numeric)
numeric_columns = ['max_sustained_wind_knots', 'min_pressure', 'wind_34kt_ne', 'wind_34kt_se', 
                  'wind_34kt_sw', 'wind_34kt_nw', 'wind_50kt_ne', 'wind_50kt_se',
                  'wind_50kt_sw', 'wind_50kt_nw', 'wind_64kt_ne', 'wind_64kt_se',
                  'wind_64kt_sw', 'wind_64kt_nw', 'max_wind_radius']

for col in numeric_columns:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')



In [16]:
# Step 12: Replace -999 with NaN (from cheatsheet: replace)

# Notes from PDF:Radius of Maximum Wind: These values have been best tracked only starting in 2021. Before 2021, 
# the missing data are denoted as “-999”. Uncertainty in the RMW values – expressed as estimated 
# absolute error in nautical miles – have been provided by a survey of the NHC Hurricane

df_clean = df_clean.replace(-999, np.nan)

df_clean.head()

,storm_id,storm_name,best_track_entry,date,time,record_identifier,storm_status,latitude,longitude,max_sustained_wind_knots,...,wind_34kt_nw,wind_50kt_ne,wind_50kt_se,wind_50kt_sw,wind_50kt_nw,wind_64kt_ne,wind_64kt_se,wind_64kt_sw,wind_64kt_nw,max_wind_radius
1,AL011851,UNNAMED,14,18510625,0000,,HU,28.0,-94.8,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AL011851,UNNAMED,14,18510625,0600,,HU,28.0,-95.4,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AL011851,UNNAMED,14,18510625,1200,,HU,28.0,-96.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AL011851,UNNAMED,14,18510625,1800,,HU,28.1,-96.5,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AL011851,UNNAMED,14,18510625,2100,L,HU,28.2,-96.8,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:

# Step 13: Create datetime (from cheatsheet: pd.to_datetime)
df_clean['datetime'] = pd.to_datetime(df_clean['date'] + df_clean['time'].str.zfill(4), 
                                      format='%Y%m%d%H%M', errors='coerce')

df_clean.head()



,storm_id,storm_name,best_track_entry,date,time,record_identifier,storm_status,latitude,longitude,max_sustained_wind_knots,...,wind_50kt_ne,wind_50kt_se,wind_50kt_sw,wind_50kt_nw,wind_64kt_ne,wind_64kt_se,wind_64kt_sw,wind_64kt_nw,max_wind_radius,datetime
1,AL011851,UNNAMED,14,18510625,0000,,HU,28.0,-94.8,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,AL011851,UNNAMED,14,18510625,0600,,HU,28.0,-95.4,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,AL011851,UNNAMED,14,18510625,1200,,HU,28.0,-96.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,AL011851,UNNAMED,14,18510625,1800,,HU,28.1,-96.5,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
5,AL011851,UNNAMED,14,18510625,2100,L,HU,28.2,-96.8,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [18]:
# Step 14: Extract year from storm_id (from cheatsheet: str slicing)
df_clean['year'] = df_clean['storm_id'].str[4:8].astype(int)
df_clean['basin'] = df_clean['storm_id'].str[:2]
df_clean['storm_number'] = df_clean['storm_id'].str[2:4].astype(int)

df_clean.head()


,storm_id,storm_name,best_track_entry,date,time,record_identifier,storm_status,latitude,longitude,max_sustained_wind_knots,...,wind_50kt_nw,wind_64kt_ne,wind_64kt_se,wind_64kt_sw,wind_64kt_nw,max_wind_radius,datetime,year,basin,storm_number
1,AL011851,UNNAMED,14,18510625,0000,,HU,28.0,-94.8,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1851,AL,1
2,AL011851,UNNAMED,14,18510625,0600,,HU,28.0,-95.4,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1851,AL,1
3,AL011851,UNNAMED,14,18510625,1200,,HU,28.0,-96.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1851,AL,1
4,AL011851,UNNAMED,14,18510625,1800,,HU,28.1,-96.5,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1851,AL,1
5,AL011851,UNNAMED,14,18510625,2100,L,HU,28.2,-96.8,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1851,AL,1


In [19]:
# Step 15: Replace basin codes with readable names (from cheatsheet: replace)
print(f"Step 15 Basin codes before replacement:")
print(df_clean['basin'].value_counts())

# Replace basin codes with full names
basin_mapping = {
    'AL': 'Atlantic',
    'EP': 'Eastern Pacific', 
    'CP': 'Central Pacific',
    'WP': 'Western Pacific'
}

df_clean['basin'] = df_clean['basin'].replace(basin_mapping)

print(f"\nStep 15 Basin names after replacement:")
print(df_clean['basin'].value_counts())

df_clean.head()

Step 15 Basin codes before replacement:
basin
AL    55230
Name: count, dtype: int64

Step 15 Basin names after replacement:
basin
Atlantic    55230
Name: count, dtype: int64


,storm_id,storm_name,best_track_entry,date,time,record_identifier,storm_status,latitude,longitude,max_sustained_wind_knots,...,wind_50kt_nw,wind_64kt_ne,wind_64kt_se,wind_64kt_sw,wind_64kt_nw,max_wind_radius,datetime,year,basin,storm_number
1,AL011851,UNNAMED,14,18510625,0000,,HU,28.0,-94.8,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1851,Atlantic,1
2,AL011851,UNNAMED,14,18510625,0600,,HU,28.0,-95.4,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1851,Atlantic,1
3,AL011851,UNNAMED,14,18510625,1200,,HU,28.0,-96.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1851,Atlantic,1
4,AL011851,UNNAMED,14,18510625,1800,,HU,28.1,-96.5,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1851,Atlantic,1
5,AL011851,UNNAMED,14,18510625,2100,L,HU,28.2,-96.8,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1851,Atlantic,1


In [20]:
# Step 16: Replace record identifier with readable names (from cheatsheet: replace)
print(f"Step 16 record identifier codes before replacement:")
print(df_clean['record_identifier'].value_counts())

# Replace record identifier codes with full names
record_code = {
    'C' : 'Closest approach to a coast, not followed by a landfall',
    'G' : 'Genesis',
    'I' : 'An intensity peak in terms of both pressure and wind',
    'L' : 'Landfall (center of system crossing a coastline)',
    'P' : 'Minimum in central pressure',
    'R' : 'Provides additional detail on the intensity of the cyclone when rapid changes are underway',
    'S' : 'Change of status of the system',
    'T' : 'Provides additional detail on the track (position) of the cyclone',
    'W' : 'Maximum sustained wind speed'
}

df_clean['record_identifier'] = df_clean['record_identifier'].replace(record_code)

print(f"\nStep 15 Record identifier names after replacement:")
print(df_clean['record_identifier'].value_counts())

df_clean.head()

Step 16 record identifier codes before replacement:
record_identifier
     53982
L     1167
I       33
R       11
P       10
T        9
S        8
C        5
W        4
G        1
Name: count, dtype: int64

Step 15 Record identifier names after replacement:
record_identifier
                                                                                              53982
Landfall (center of system crossing a coastline)                                               1167
An intensity peak in terms of both pressure and wind                                             33
Provides additional detail on the intensity of the cyclone when rapid changes are underway       11
Minimum in central pressure                                                                      10
Provides additional detail on the track (position) of the cyclone                                 9
Change of status of the system                                                                    8
Closest approach to a co

,storm_id,storm_name,best_track_entry,date,time,record_identifier,storm_status,latitude,longitude,max_sustained_wind_knots,...,wind_50kt_nw,wind_64kt_ne,wind_64kt_se,wind_64kt_sw,wind_64kt_nw,max_wind_radius,datetime,year,basin,storm_number
1,AL011851,UNNAMED,14,18510625,0000,,HU,28.0,-94.8,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1851,Atlantic,1
2,AL011851,UNNAMED,14,18510625,0600,,HU,28.0,-95.4,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1851,Atlantic,1
3,AL011851,UNNAMED,14,18510625,1200,,HU,28.0,-96.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1851,Atlantic,1
4,AL011851,UNNAMED,14,18510625,1800,,HU,28.1,-96.5,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1851,Atlantic,1
5,AL011851,UNNAMED,14,18510625,2100,Landfall (center of system crossing a coastline),HU,28.2,-96.8,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1851,Atlantic,1


In [21]:
# Step 17: Replace storm status codes with readable names (from cheatsheet: replace)
print(f"Step 17 Storm status codes before replacement:")
print(df_clean['storm_status'].value_counts())

# Replace storm status with full names
storm_status = {
    'TD' : 'Tropical cyclone of tropical depression intensity (< 34 knots)',
    'TS' : 'Tropical cyclone of tropical storm intensity (34-63 knots)',
    'HU' : 'Tropical cyclone of hurricane intensity (> 64 knots)',
    'EX' : 'Extratropical cyclone (of any intensity)',
    'SD' : 'Subtropical cyclone of subtropical depression intensity (< 34 knots)',
    'SS' : 'Subtropical cyclone of subtropical storm intensity (> 34 knots)',
    'LO' : 'A low that is neither a tropical cyclone, a subtropical cyclone, nor an extratropical cyclone (of any intensity)',
    'WV' : 'Tropical Wave (of any intensity)',
    'DB' : 'Disturbance (of any intensity)'
}

df_clean['storm_status'] = df_clean['storm_status'].replace(storm_status)

print(f"\nStep 17 Storm status names after replacement:")
print(df_clean['storm_status'].value_counts())

df_clean.head()

Step 17 Storm status codes before replacement:
storm_status
TS    20246
HU    15684
TD     9914
EX     6189
LO     1704
SS      730
SD      326
DB      299
WV      138
Name: count, dtype: int64

Step 17 Storm status names after replacement:
storm_status
Tropical cyclone of tropical storm intensity (34-63 knots)                                                          20246
Tropical cyclone of hurricane intensity (> 64 knots)                                                                15684
Tropical cyclone of tropical depression intensity (< 34 knots)                                                       9914
Extratropical cyclone (of any intensity)                                                                             6189
A low that is neither a tropical cyclone, a subtropical cyclone, nor an extratropical cyclone (of any intensity)     1704
Subtropical cyclone of subtropical storm intensity (> 34 knots)                                                       730
Subtropical cy

,storm_id,storm_name,best_track_entry,date,time,record_identifier,storm_status,latitude,longitude,max_sustained_wind_knots,...,wind_50kt_nw,wind_64kt_ne,wind_64kt_se,wind_64kt_sw,wind_64kt_nw,max_wind_radius,datetime,year,basin,storm_number
1,AL011851,UNNAMED,14,18510625,0000,,Tropical cyclone of hurricane intensity (> 64 ...,28.0,-94.8,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1851,Atlantic,1
2,AL011851,UNNAMED,14,18510625,0600,,Tropical cyclone of hurricane intensity (> 64 ...,28.0,-95.4,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1851,Atlantic,1
3,AL011851,UNNAMED,14,18510625,1200,,Tropical cyclone of hurricane intensity (> 64 ...,28.0,-96.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1851,Atlantic,1
4,AL011851,UNNAMED,14,18510625,1800,,Tropical cyclone of hurricane intensity (> 64 ...,28.1,-96.5,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1851,Atlantic,1
5,AL011851,UNNAMED,14,18510625,2100,Landfall (center of system crossing a coastline),Tropical cyclone of hurricane intensity (> 64 ...,28.2,-96.8,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,1851,Atlantic,1


In [22]:
# Step 18: Create storm category column based on Saffir-Simpson scale
print(f"Step 18 - Adding storm category based on max sustained wind (knots)")

def categorize_storm(wind_knots):
    """
    Categorize storm based on Saffir-Simpson scale using wind speed in knots
    """
    if pd.isna(wind_knots):
        return 'Unknown'
    elif wind_knots <= 33:
        return 'TD'  # Tropical Depression
    elif 34 <= wind_knots <= 63:
        return 'TS'  # Tropical Storm
    elif 64 <= wind_knots <= 82:
        return 'Category 1'
    elif 83 <= wind_knots <= 95:
        return 'Category 2'
    elif 96 <= wind_knots <= 112:
        return 'Category 3'
    elif 113 <= wind_knots <= 136:
        return 'Category 4'
    elif wind_knots >= 137:
        return 'Category 5'
    else:
        return 'Unknown'

# Apply the categorization function
df_clean['storm_category'] = df_clean['max_sustained_wind_knots'].apply(categorize_storm)

print(f"\nStorm category distribution:")
print(df_clean['storm_category'].value_counts().sort_index())

# Show some examples
print(f"\nExamples of storm categorization:")
sample_data = df_clean[['storm_name', 'max_sustained_wind_knots', 'storm_category']].dropna()
print(sample_data.head(10))

# Saffir-Simpson Hurricane Wind Scale source: https://www.nhc.noaa.gov/aboutsshws.php

Step 18 - Adding storm category based on max sustained wind (knots)

Storm category distribution:
storm_category
Category 1     8919
Category 2     4090
Category 3     2064
Category 4     1093
Category 5      188
TD            13221
TS            25655
Name: count, dtype: int64

Examples of storm categorization:
   storm_name  max_sustained_wind_knots storm_category
1     UNNAMED                        80     Category 1
2     UNNAMED                        80     Category 1
3     UNNAMED                        80     Category 1
4     UNNAMED                        80     Category 1
5     UNNAMED                        80     Category 1
6     UNNAMED                        70     Category 1
7     UNNAMED                        60             TS
8     UNNAMED                        60             TS
9     UNNAMED                        50             TS
10    UNNAMED                        50             TS


In [23]:

# Step 19: Reset index (from cheatsheet: reset_index)
df_clean = df_clean.reset_index(drop=True)

In [24]:
# Step 20: Save to CSV
df_clean.to_csv('/home/julianac/200-Fall2025-EDA-Team-Hurricane/cleaned_hurricane_data.csv', index=False)
print(f"✓ Saved cleaned data to: /home/julianac/200-Fall2025-EDA-Team-Hurricane/cleaned_hurricane_data.csv")
print(f"Total rows: {len(df_clean)}")
print(f"Total columns: {len(df_clean.columns)}")

✓ Saved cleaned data to: /home/julianac/200-Fall2025-EDA-Team-Hurricane/cleaned_hurricane_data.csv
Total rows: 55230
Total columns: 29


# Definitions

Hi Team, We are using the following units, we can discuss
    -hemisphere: this is implied with negative coordinates
    -Lat, lon
    -nautical miles
    -wind: knots which maybe we should convert to mph 
    -time: UTC, year, month, day, hours, mins 

    There are two types of lines of data in the new format: the *header line* and the *data lines*. 
    The format is comma delimited to maximize its ease in use. 

### The header line has the following format:
    AL092021,
    IDA,
    40,
    1234567890123456789012345768901234567
    AL (Spaces 1 and 2) – Basin – Atlantic
    09 (Spaces 3 and 4) – ATCF cyclone number for that year
    2021 (Spaces 5-8, before first comma) – Year
    IDA (Spaces 19-28, before second comma) – Name, if available, or else “UNNAMED”
    40 (Spaces 34-36) – Number of best track entries – rows – to follow

## Notes:
### *1) Cyclone number:* 
        In HURDAT2, the order cyclones appear in the file is determined by the date/time of the first tropical or subtropical cyclone
        record in the best track. This sequence may or may not correspond to the ATCF cyclone number. For example, the 2011 unnamed tropical storm
        AL20 which formed on 1 September, is sequenced here between AL12 (Katia – formed on 29 Aug) and AL13 (Lee – formed on 2 September).
        This mismatch between ATCF cyclone number and the HURDAT2 sequencing can occur if post-storm analysis alters the relative genesis times
        between two cyclones. In addition, in 2011 it became practice to assign operationally unnamed cyclones ATCF numbers from the end of the list,
        rather than insert them in sequence and alter the ATCF numbers of cyclones previously assigned.
### *2) Name:* 
        Tropical cyclones were not formally named before 1950 and are thus referred to as “UNNAMED” in the database. Systems that were
        added into the database after the season (such as AL20 in 2011) also are considered “UNNAMED”. Non-developing tropical depressions formallywere given names (actually numbers, such as “TEN”) that were included into the ATCF b-decks starting in 2003. Non-developing tropical
        depressions before this year are also referred to as “UNNAMED”.
        The remaining rows of data in the new format are the data lines. These have the following format:
        20210829, 1655, L, HU, 29.1N, 90.2W, 130, 931, 130, 110,
        80, 110,
        70,
        60,
        40,
        60,
        45,
        35,
        20,
        30,
        10
        12345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345
        2021 (Spaces 1-4) – Year
        08 (Spaces 5-6) – Month
        29 (Spaces 7-8, before 1st comma) – Day
        16 (Spaces 11-12) – Hours in UTC (Universal Time Coordinate)
        55 (Spaces 13-14, before 2nd comma) – Minutes
        L (Space 17, before 3rd comma) – Record identifier (see notes below)
        C – Closest approach to a coast, not followed by a landfall
        G – Genesis
        I – An intensity peak in terms of both pressure and wind
        L – Landfall (center of system crossing a coastline)
        P – Minimum in central pressure
        R – Provides additional detail on the intensity of the cyclone when rapid changes are underway
        S – Change of status of the system
        T – Provides additional detail on the track (position) of the cyclone
        W – Maximum sustained wind speed
        HU (Spaces 20-21, before 4th comma) – Status of system. Options are:
        TD – Tropical cyclone of tropical depression intensity (< 34 knots)
        TS – Tropical cyclone of tropical storm intensity (34-63 knots)
        HU – Tropical cyclone of hurricane intensity (> 64 knots)
        EX – Extratropical cyclone (of any intensity)
        SD – Subtropical cyclone of subtropical depression intensity (< 34 knots)
        SS – Subtropical cyclone of subtropical storm intensity (> 34 knots)
        LO – A low that is neither a tropical cyclone, a subtropical cyclone, nor an extratropical cyclone (of any intensity)
        WV – Tropical Wave (of any intensity)
        DB – Disturbance (of any intensity)
        29.1 (Spaces 24-27) – Latitude
        N (Space 28, before 5th comma) – Hemisphere – North or South
        90.2 (Spaces 31-35) – Longitude
        W (Space 36, before 6th comma) – Hemisphere – West or East
        130 (Spaces 39-41, before 7th comma) – Maximum sustained wind (in knots)931 (Spaces 44-47, before 8th comma) – Minimum Pressure (in millibars)
        130 (Spaces 50-53, before 9th comma) – 34 kt wind radii maximum extent in northeastern quadrant (in nautical miles)
        110 (Spaces 56-59, before 10th comma) – 34 kt wind radii maximum extent in southeastern quadrant (in nautical miles)
        70 (Spaces 62-65, before 11th comma) – 34 kt wind radii maximum extent in southwestern quadrant (in nautical miles)
        60 (Spaces 68-71, before 12th comma) – 34 kt wind radii maximum extent in northwestern quadrant (in nautical miles)
        40 (Spaces 74-77, before 13th comma) – 50 kt wind radii maximum extent in northeastern quadrant (in nautical miles)
        60 (Spaces 80-83, before 14th comma) – 50 kt wind radii maximum extent in southeastern quadrant (in nautical miles)
        80 (Spaces 86-89, before 15th comma) – 50 kt wind radii maximum extent in southwestern quadrant (in nautical miles)
        30 (Spaces 92-95, before 16th comma) – 50 kt wind radii maximum extent in northwestern quadrant (in nautical miles)
        45 (Spaces 98-101, before 17th comma) – 64 kt wind radii maximum extent in northeastern quadrant (in nautical miles)
        25 (Spaces 104-107, before 18th comma) – 64 kt wind radii maximum extent in southeastern quadrant (in nautical miles)
        35 (Spaces 110-113, before 19th comma) – 64 kt wind radii maximum extent in southwestern quadrant (in nautical miles)
        20 (Spaces 116-119, before 20th comma) – 64 kt wind radii maximum extent in northwestern quadrant (in nautical miles)
        15 (Spaces 122-125) – Radius of Maximum Wind (in nautical miles)